<a href="https://colab.research.google.com/github/tazar09/heroes_3/blob/main/h3_heroes%5B26.oct2023%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [353]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
df1 = pd.read_html('https://heroes.thelazy.net/index.php/List_of_heroes_(HotA)')

In [354]:
df1[0].head()

,Name,Class,Unnamed: 2,Specialty,Unnamed: 4,Skill 1,Unnamed: 6,Skill 2,Unnamed: 8,Spell
0,Anabel,Captain,NaN,Pirates,NaN,Basic Offense,NaN,Basic Archery,NaN,NaN
1,Cassiopeia,Captain,NaN,Nymphs,NaN,Basic Offense,NaN,Basic Tactics,NaN,NaN
2,Corkes,Captain,NaN,Offense,NaN,Basic Offense,NaN,Basic Pathfinding,NaN,NaN
3,Derek,Captain,NaN,Crew Mates,NaN,Basic Offense,NaN,Basic Leadership,NaN,NaN
4,Elmore,Captain,NaN,Navigation,NaN,Advanced Navigation,NaN,NaN,NaN,NaN


In [355]:
df = df1[0].copy()
df = df.drop(axis = 1, labels = ['Unnamed: 2','Unnamed: 4', 'Unnamed: 6', 'Unnamed: 8'])
df.columns = ['name', 'class', 'specialty', 'skill_1', 'skill_2', 'spell']

In [356]:
skill1 = df['skill_1'].str.split(expand = True, n = 1)
skill2 = df['skill_2'].str.split(expand = True, n = 1)
df = pd.concat([df, skill1, skill2], axis = 1)
df.columns = ['name', 'class', 'specialty', 'skill_1', 'skill_2', 'spell', 'skill_1_level', 'skill_1_name', 'skill_2_level', 'skill_2_name']

In [357]:
df = df.drop(axis = 1, labels = ['skill_1', 'skill_2'])

In [358]:
df = df[['name', 'class', 'specialty', 'skill_1_name', 'skill_1_level', 'skill_2_name','skill_2_level', 'spell']]
df['spell'] = df['spell'].fillna('')
df['skill_2_name'] = df['skill_2_name'].fillna('')
df['skill_2_level'] = df['skill_2_level'].fillna('')
df['count'] = 1
df['spell_tf'] = df['spell'].apply(lambda x: 0 if x == '' else 1)
df.head()

,name,class,specialty,skill_1_name,skill_1_level,skill_2_name,skill_2_level,spell,count,spell_tf
0,Anabel,Captain,Pirates,Offense,Basic,Archery,Basic,,1,0
1,Cassiopeia,Captain,Nymphs,Offense,Basic,Tactics,Basic,,1,0
2,Corkes,Captain,Offense,Offense,Basic,Pathfinding,Basic,,1,0
3,Derek,Captain,Crew Mates,Offense,Basic,Leadership,Basic,,1,0
4,Elmore,Captain,Navigation,Navigation,Advanced,,,,1,0


In [359]:
towns = pd.read_html('http://heroes.thelazy.net/index.php/List_of_all_town_names')
towns = list(towns[1]['Town'].unique())

In [360]:
towns = ['Castle', 'Rampart', 'Tower', 'Inferno', 'Necropolis','Dungeon', 'Stronghold' ,'Fortress','Conflux', 'Cove']

In [361]:
classes = list(df['class'].unique())
classes.remove('Captain')
classes.remove('Navigator')
classes.insert(18, 'Captain')
classes.insert(19, 'Navigator')
classes[8] = 'Death Knight'

In [362]:
towns2 = []
temp_list = []
for i in towns:
  temp_list = [i, i]
  towns2.append(temp_list)
towns2x = [element for sublist in towns2 for element in sublist]
len(classes), len(towns2x)

(20, 20)

In [363]:
class_towns = pd.DataFrame({'class': classes, 'town': towns2x})
df = df.merge(class_towns, how='left', on='class')

In [364]:
df.columns = ['name', 'class', 'specialty', 'skill_1', 'level_1',
       'skill_2', 'level_2', 'spell', 'count', 'spell_tf', 'town']

In [365]:
df['class_2'] = df['skill_1'].apply(lambda x: 'magic' if x == 'Wisdom' else 'might')
df.iloc[81, 11] = 'might'  #Rashka
df.iloc[101, 11] = 'magic' #Aislinn

In [366]:
df = df[['name', 'class', 'class_2', 'specialty', 'level_1', 'skill_1',
       'level_2', 'skill_2', 'spell', 'count', 'spell_tf', 'town']]

In [367]:
df.iloc[91:101 , 11] = 'Necropolis'

In [368]:
creatures = pd.read_csv('/content/df_h3.csv')
creatures = creatures[['Name', 'Town','Upgraded']]

In [369]:
creatures['specialty'] = creatures['Name'].apply(lambda x: x + 's')
creatures = creatures[creatures['Upgraded'] == 0].reset_index(drop = True)

In [370]:
creatures['specialty'] = creatures['specialty'].apply(lambda x: 'convert' if x.endswith('mans') else x)

In [371]:
# creatures[creatures['specialty'] == 'convert']
creatures.iloc[0, 3] = 'Pikemen'
creatures.iloc[3, 3] = 'Swordsmen'
creatures.iloc[50, 3] = 'Lizardmen'

creatures.iloc[64, 3] = 'Crew Mate'
creatures.iloc[65, 3] = 'Sea Dogs'
creatures.iloc[67, 3] = 'Sea Witches'
creatures.iloc[68, 3] = 'Nix'

creatures.iloc[8, 3] = 'Dwarves'
creatures.iloc[9, 3] = 'Elves'
creatures.iloc[10, 3] = 'Pegasi'
creatures.iloc[26, 3] = 'Efreet'
creatures.iloc[32, 3] = 'Liches'
creatures.iloc[32, 3] = 'Liches'
creatures.iloc[36, 3] = 'Harpies'
creatures.iloc[47, 3] = 'Cyclops'
creatures.iloc[51, 3] = 'Cyclops'

creatures.iloc[11, 3] = 'Dendroids'
creatures.iloc[16, 3] = 'Golems'
creatures.iloc[17, 3] = 'Magi'

# creatures.head(5)

In [413]:
df1 = df.merge(creatures, how = 'left', on = 'specialty', suffixes = None)
df1 = df1.drop(axis = 0, index = 135).reset_index(drop = True)

In [414]:
df1.iloc[73,12] = 'Enchanter'
df1.iloc[0,12] = 'Pirate'
df1.iloc[3,12] = 'Crew Mate'
df1.iloc[9,12] = 'Sea Dog'
df1.iloc[96,12] = 'Walking Dead'
df1.iloc[150, 12] = 'Serpent Fly'
df1 = df1.drop(axis = 0, index = 135).reset_index(drop = True)

In [ ]:
df1['Name'] = df1['Name'].fillna('')
df1 = df1.drop(axis=1, labels = ['Town', 'Upgraded'])


In [420]:
df1[df1['Name'] == ''][['name', 'specialty']]

,name,specialty
2,Corkes,Offense
4,Elmore,Navigation
6,Jeremy,Cannon
7,Leena,Gold
8,Miriam,Scouting
11,Andal,Crystal
12,Astra,Cure
14,Dargem,Air Shield
15,Eovacius,Clone
16,Manfred,Fireball


In [397]:
df[df['town'] == 'Cove'][['specialty']].head(10)

,specialty
0,Pirates
1,Nymphs
2,Offense
3,Crew Mates
4,Navigation
5,Stormbirds
6,Cannon
7,Gold
8,Scouting
9,Sea Dogs


In [398]:
# df[~(df['Name'] == '')][['name', 'class', 'specialty','Name', 'Town']].head()

In [399]:
# df['Name'] = df['Name'].fillna('')
# df[df['Town'] == 'Cove'][['name', 'specialty', 'Name']].head(200)